# CLARISSA Code Examples

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rXPnEVc0du1su8Splil-8okX6kESwXI7)

**Click the badge above to run this notebook interactively in Google Colab!**

Executable Python code from the [Reservoir Basics Tutorial](https://irena-40cc50.gitlab.io/tutorials/reservoir-basics/).

---
## Setup (Team Members)

Run the cell below to connect to GitLab. Credentials are loaded automatically from the shared GDrive folder.

**First-time setup:** Add the shared [BLAUWEISS-EDV-LLC folder](https://drive.google.com/drive/folders/1qh0skTeyRNs4g9KwAhpd3J8Yj_XENIFs) to your GDrive:
1. Open the link above
2. Right-click the folder > **Organize** > **Add shortcut to Drive**
3. Done! Credentials will load automatically.

In [ ]:
#@title Mount GDrive & Clone Repository { display-mode: "form" }
#@markdown Credentials loaded from BLAUWEISS-EDV-LLC/CLARISSA/config folder.

import os
import json

IN_COLAB = False
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except ImportError:
    print('Not in Colab - running locally')

# Possible locations for credentials file (ADR-017 structure)
CRED_PATHS = [
    # Shortcut at root
    '/content/drive/MyDrive/BLAUWEISS-EDV-LLC/CLARISSA/config/clarissa_credentials.json',
    # Shared drive
    '/content/drive/Shareddrives/BLAUWEISS-EDV-LLC/CLARISSA/config/clarissa_credentials.json',
    # Legacy paths (fallback)
    '/content/drive/MyDrive/CLARISSA/config/clarissa_credentials.json',
    '/content/drive/MyDrive/clarissa_credentials.json',
]

creds = None
if IN_COLAB:
    for path in CRED_PATHS:
        if os.path.exists(path):
            with open(path, 'r') as f:
                creds = json.load(f)
            print(f'Credentials loaded from: {path}')
            break
    
    if not creds:
        print('Credentials not found!')
        print('Add BLAUWEISS-EDV-LLC folder to your GDrive:')
        print('https://drive.google.com/drive/folders/1qh0skTeyRNs4g9KwAhpd3J8Yj_XENIFs')

# Clone repository
REPO_DIR = '/content/irena'

if creds and not os.path.exists(REPO_DIR):
    token = creds['gitlab']['pat']
    repo = creds['gitlab']['repo_url'].replace('https://', f'https://oauth2:{token}@')
    !git clone {repo} {REPO_DIR}
    print(f'Cloned to {REPO_DIR}')
elif os.path.exists(REPO_DIR):
    !cd {REPO_DIR} && git pull

if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print(f'Working in: {os.getcwd()}')

---
## Pore Volume Calculation

In [ ]:
# Reservoir parameters (SPE9 model)
nx, ny, nz = 24, 25, 15  # Grid dimensions
dx, dy, dz = 300, 300, 50  # Cell sizes in feet
porosity = 0.087  # Average porosity
ntg = 1.0  # Net-to-gross ratio

# Calculate volumes
bulk_volume = nx * ny * nz * dx * dy * dz  # ft3
pore_volume = bulk_volume * porosity * ntg
pore_volume_bbl = pore_volume / 5.615  # Convert to barrels

print(f'Grid: {nx} x {ny} x {nz} = {nx*ny*nz:,} cells')
print(f'Pore volume: {pore_volume_bbl/1e6:.2f} MMbbl')

## Material Balance - OOIP

In [ ]:
def calculate_ooip(np_cum, bo, boi, delta_p, ce):
    """Calculate OOIP using simplified material balance."""
    return (np_cum * bo) / (boi * ce * delta_p)

N = calculate_ooip(
    np_cum=1_500_000,  # STB produced
    bo=1.25,
    boi=1.30,
    delta_p=500,  # psi
    ce=15e-6  # 1/psi
)

print(f'Estimated OOIP: {N/1e6:.1f} MMSTB')

## Recovery Factor Estimation

In [ ]:
drive_mechanisms = {
    'Solution gas drive': (0.05, 0.30),
    'Gas cap drive': (0.20, 0.40),
    'Water drive': (0.35, 0.75),
    'Gravity drainage': (0.50, 0.70),
}

ooip_mmstb = 150

print(f'OOIP: {ooip_mmstb} MMSTB')
print()
print(f'{"Drive Mechanism":<25} {"RF Range":>15} {"Recoverable (MMSTB)":>20}')
print('-' * 62)

for mechanism, (rf_low, rf_high) in drive_mechanisms.items():
    rec_low = ooip_mmstb * rf_low
    rec_high = ooip_mmstb * rf_high
    print(f'{mechanism:<25} {rf_low*100:>5.0f}% - {rf_high*100:<5.0f}% {rec_low:>8.1f} - {rec_high:<8.1f}')

---
## Save Changes to GitLab

In [ ]:
#@title Commit & Push Changes { display-mode: "form" }
commit_message = "docs(notebook): update from Colab"  #@param {type:"string"}

import os
if os.path.exists('/content/irena/.git'):
    os.chdir('/content/irena')
    !git config user.email "colab@clarissa.dev"
    !git config user.name "CLARISSA Colab"
    !git add -A
    !git status
    !git commit -m "{commit_message}" || echo "Nothing to commit"
    !git push
else:
    print('Run setup cell first')